# SQL3 ACTIONS PONTUELLES

In [1]:
import sqlite3

## TRANSFERTS DE TABLES

In [4]:
def tbl_tranfert(source_db_path, destination_db_path, table_to_copy):
    source_connection = sqlite3.connect(source_db_path)
    source_cursor = source_connection.cursor()
    destination_connection = sqlite3.connect(destination_db_path)
    destination_cursor = destination_connection.cursor()

    # Step 1: Fetch the structure of the source table
    source_cursor.execute(f'PRAGMA table_info("{table_to_copy}")')
    columns = source_cursor.fetchall()
    column_names = [column[1] for column in columns]
    column_definitions = ', '.join([f'"{column[1]}" {column[2]}' for column in columns])

    # Step 2: Create the destination table
    destination_cursor.execute(f'CREATE TABLE IF NOT EXISTS "{table_to_copy}" ({column_definitions})')

    # Step 3: Fetch data from the source database
    source_cursor.execute(f'SELECT * FROM "{table_to_copy}"')
    data_to_copy = source_cursor.fetchall()

    # Step 4: Insert data into the destination database
    placeholders = ', '.join(['?' for _ in range(len(columns))])
    destination_cursor.executemany(f'INSERT INTO "{table_to_copy}" VALUES ({placeholders})', data_to_copy)

    # Commit the changes to the destination database
    destination_connection.commit()

    # Close both connections
    source_connection.close()
    destination_connection.close()


In [23]:
# Source database connection
source_db_path = 'HistoPlotik-LGB.db'
# Destination database connection
destination_db_path = 'HistoPlotik.db'

In [4]:
# tbl_tranfert(source_db_path, destination_db_path, 'LGB_reg200_importance_tbl')

In [5]:
# tbl_tranfert(source_db_path, destination_db_path, 'LGB_reg200_resultats_tbl')

## RENAME OLD TABLE

In [2]:
db_path = 'HistoPlotik.db'
# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Rename the table
old_table_name = 'RandomF_reg200_resultats_tbl'
new_table_name = 'RandomF_reg200_resultats_tbl_OLD'
query = f'ALTER TABLE {old_table_name} RENAME TO {new_table_name}'
cursor.execute(query)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Rename the table
old_table_name = 'RandomF_reg200_importance_tbl'
new_table_name = 'RandomF_reg200_importance_tbl_OLD'
query = f'ALTER TABLE {old_table_name} RENAME TO {new_table_name}'
cursor.execute(query)

# Commit the changes and close the connection
conn.commit()
conn.close()

## CREATION TABLE DEPUIS DF
Anouar

In [11]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np


In [28]:
db_path = 'HistoPlotik.db'
# Connect to the SQLite database (replace 'your_database.db' with your actual database file)
connection = sqlite3.connect(db_path)

tbl_db_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)
for table in tbl_db_names['name'] :
    print (table)

# Close the database connection
connection.close()

XGBoost_reg200_resultats_tbl
XGBoost_reg200_importance_tbl
y_resultats200_tbl
X_col_titr
data_tbl
LGB_reg200_importance_tbl
LGB_reg200_resultats_tbl
villes_tbl
RandomF_reg200_resultats_tbl_OLD
RandomF_reg200_importance_tbl_OLD
RandomF_reg200_resultats_tbl


### RandomF_reg200_resultats_tbl

In [17]:
file_path='../MODELS/IMPORT_MODELS/df_result_RF_optim_GridSearch.csv'
df_RandomF_res= pd.read_csv(file_path, encoding='utf-8', sep=';', index_col=0, low_memory=False)
df_RandomF_res.shape

(23946, 7)

In [18]:
df_RandomF_res.head(2)

,pred_YYY_pvoixNUP,pred_YYY_pvoixECO,pred_YYY_pvoixENS,pred_YYY_pvoixLR_UDI,pred_YYY_pvoixEXTREMD,pred_YYY_pvoixCENTRE,pred_YYY_pvoixOTR
key,,,,,,,
01001,0.178214,0.023036,0.174325,0.255469,0.291478,0.064350,0.016327
01002,0.318701,0.020174,0.095429,0.161103,0.169906,0.330483,0.033145


In [19]:
from sqlalchemy import create_engine, MetaData, Table

# Remplacez 'your_database_url' par l'URL de connexion à votre base de données
engine = create_engine(f"sqlite:///{db_path}")

# Create a MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

In [20]:
tabl_export='RandomF_reg200_resultats_tbl2'
if tabl_export not in tbl_db_names['name'] :
    # Convert DataFrame to a SQL table, using the index as the ID column
    df_RandomF_res.to_sql(tabl_export, con=engine, index=True, if_exists='replace')
    metadata.reflect(bind=engine)
tbl_db_names = metadata.tables.keys()
tbl_db_names

dict_keys(['LGB_reg200_importance_tbl', 'LGB_reg200_resultats_tbl', 'RandomF_reg200_importance_tbl_OLD', 'RandomF_reg200_resultats_tbl', 'RandomF_reg200_resultats_tbl_OLD', 'XGBoost_reg200_importance_tbl', 'XGBoost_reg200_resultats_tbl', 'X_col_titr', 'data_tbl', 'villes_tbl', 'y_resultats200_tbl', 'RandomF_reg200_resultats_tbl2'])

In [22]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Rename the table
old_table_name = 'RandomF_reg200_resultats_tbl2'
new_table_name = 'RandomF_reg200_resultats_tbl'

# Drop the existing table with the new name, if it exists
cursor.execute(f'DROP TABLE IF EXISTS {new_table_name}')

query = f'ALTER TABLE {old_table_name} RENAME TO {new_table_name}'
cursor.execute(query)

# Commit the changes and close the connection
conn.commit()
conn.close()

### RandomF_reg200_importance_tbl

In [23]:
# file_path='../MODELS/IMPORT_MODELS/feature_importance_df_Anouar.csv'
file_path='../MODELS/IMPORT_MODELS/feature_importance_df_RF_optim_GridSearch.csv'
df_RandomF_FI= pd.read_csv(file_path, encoding='utf-8', sep=';', index_col=0, low_memory=False)
df_RandomF_FI.shape

(261, 14)

In [24]:
df_RandomF_FI.head(2)

,YYY_pvoixNUP_estim_indice,YYY_pvoixNUP_import,YYY_pvoixECO_estim_indice,YYY_pvoixECO_import,YYY_pvoixENS_estim_indice,YYY_pvoixENS_import,YYY_pvoixLR_UDI_estim_indice,YYY_pvoixLR_UDI_import,YYY_pvoixEXTREMD_estim_indice,YYY_pvoixEXTREMD_import,YYY_pvoixCENTRE_estim_indice,YYY_pvoixCENTRE_import,YYY_pvoixOTR_estim_indice,YYY_pvoixOTR_import
0,127,0.157043,127,0.078286,127,0.139526,126,0.206866,126,0.173471,126,0.188561,127,0.636271
1,126,0.087769,212,0.077490,126,0.072486,127,0.188310,127,0.122494,127,0.180958,126,0.095048


In [25]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current Directory:", current_directory)

Current Directory: /home/nicolas/code/nyxibe/HistoPolitik_2/DATA


In [26]:
from sqlalchemy import create_engine, MetaData, Table

# Remplacez 'your_database_url' par l'URL de connexion à votre base de données
engine = create_engine(f"sqlite:///{db_path}")

# Create a MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

In [30]:
tabl_export='RandomF_reg200_importance_tbl2'
if tabl_export not in tbl_db_names['name'] :
    # Convert DataFrame to a SQL table, using the index as the ID column
    df_RandomF_FI.to_sql(tabl_export, con=engine, index=True, if_exists='replace')
    metadata.reflect(bind=engine)
tbl_db_names = metadata.tables.keys()
tbl_db_names

dict_keys(['LGB_reg200_importance_tbl', 'LGB_reg200_resultats_tbl', 'RandomF_reg200_importance_tbl_OLD', 'RandomF_reg200_resultats_tbl', 'RandomF_reg200_resultats_tbl_OLD', 'XGBoost_reg200_importance_tbl', 'XGBoost_reg200_resultats_tbl', 'X_col_titr', 'data_tbl', 'villes_tbl', 'y_resultats200_tbl', 'RandomF_reg200_importance_tbl', 'RandomF_reg200_importance_tbl2'])

In [31]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Rename the table
old_table_name = 'RandomF_reg200_importance_tbl2'
new_table_name = 'RandomF_reg200_importance_tbl'

# Drop the existing table with the new name, if it exists
cursor.execute(f'DROP TABLE IF EXISTS {new_table_name}')

query = f'ALTER TABLE {old_table_name} RENAME TO {new_table_name}'
cursor.execute(query)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [32]:
connection = sqlite3.connect(db_path)

tbl_db_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)
for table in tbl_db_names['name'] :
    print (table)

# Close the database connection

XGBoost_reg200_resultats_tbl
XGBoost_reg200_importance_tbl
y_resultats200_tbl
X_col_titr
data_tbl
LGB_reg200_importance_tbl
LGB_reg200_resultats_tbl
villes_tbl
RandomF_reg200_resultats_tbl_OLD
RandomF_reg200_importance_tbl_OLD
RandomF_reg200_resultats_tbl
RandomF_reg200_importance_tbl
